# Un modèle prédictif amélioré

Dans le tout premier TD, vous avez programmé un modèle d’apprentissage supervisé pour une tâche de classification de manchots de l’Antarctique. Depuis, vous avez appris davantage de techniques, comme la manière de préparer convenablement ses données, aussi votre objectif sera de parvenir à programmer un modèle qui produira de meilleures prédictions.

## Le pré-traitement des données

### Préparer le *dataset*

La toute première étape est de charger les données dans un *data frame* :

In [ ]:
# your code here

import pandas as pd

df = pd.read_csv("../data/penguin-census.csv")

Définissez maintenant la variable cible pour la prédiction, ainsi que les variables explicatives, puis constituez vos structures `X` et `y` avec, respectivement, les variables explicatives et la variable cible :

In [ ]:
# your code here

target = "species"
features = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g", "island", "sex"]

X = df[features]
y = df[target]

Distinguez dans `X` les variables qualitatives et quantitatives grâce à la fonction `make_column_selector()` :

In [ ]:
# your code here

from sklearn.compose import make_column_selector as col_selector

num_col_selector = col_selector(dtype_exclude=object)
cat_col_selector = col_selector(dtype_include=object)

num_cols = num_col_selector(X)
cat_cols = cat_col_selector(X)

### Préparer les variables quantitatives

Le *data frame* qui contient les variables explicatives est maintenant constitué. Vous allez pas à pas construire toute une structure pour transformer les données de manière à les envoyer au modèle d’apprentissage choisi.

Commencez par les variables quantitatives et mettez en place un pipeline pour :
1. remplacer les données manquantes avec un `SimpleImputer` ;
2. mettre les données à l’échelle avec un `StandardScaler`.

In [ ]:
# your code here

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

num_transformers = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

### Préparer les variables qualitatives

Faites de même pour les variables qualitatives en remplaçant également les données manquantes avec un `SimpleImputer` puis en les encodant avec un `OneHotEncoder` :

In [ ]:
# your code here

from sklearn.preprocessing import OneHotEncoder

cat_transformers = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="?"),
    OneHotEncoder(handle_unknown="ignore")
)

### Assembler les transformateurs

Dans une variable `preprocessor`, préparez un `ColumnTransformer` pour recueillir vos transformateurs numériques et catégorielles :

In [ ]:
# your code here

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num-transformers", num_transformers, num_cols),
        ("cat-transformers", cat_transformers, cat_cols),
    ]
)

## Créer le modèle d’apprentissage

Pour les besoins de votre tâche, vous utiliserez un modèle linéaire appelé `LogisticRegression`. Constituez un nouveau pipeline qui assemble les transformateurs contenus dans votre variable `preprocessor` et le modèle de régression logistique :

In [ ]:
# your code here

from sklearn.linear_model import LogisticRegression

model = make_pipeline(preprocessor, LogisticRegression())

Affichez votre modèle pour bien en comprendre le fonctionnement :

In [ ]:
model

## Ajuster le modèle

Il ne reste plus qu’à entraîner votre modèle et à le tester sur votre jeu de données.

Séparez tout d’abord vos données en jeux d’entraînement et de test :

In [ ]:
# your code here

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Grâce à la méthode `.fit()` entraînez votre modèle :

In [ ]:
# your code here

_ = model.fit(X_train, y_train)

Dernière étape, faites des prédictions avec la méthode `.predict()` :

In [ ]:
# your code here

y_pred = model.predict(X_test)

## Évaluer le modèle

Votre modèle est prêt et il a effectué des prédictions pour le jeu de test. Mais quelle est sa précision ? Donnez les résultats pour le jeu d’entraînement et pour le jeu de test :

In [ ]:
# your code here

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(
    f"Score de précision sur le jeu d’entraînement : {train_score:.3f}",
    f"Score de précision sur le jeu de test : {test_score:.3f}",
    sep="\n"
)

Si vous avez bien respecté les étapes, les deux scores devraient être très élevés, voire atteindre 100 %. Lorsque l’on obtient des résultats aussi encourageants, c’est souvent le symptôme d’un modèle sur-entraîné. Essayez d’expliquer pourquoi le modèle s’ajuste aussi bien aux données.